In [1]:
import pandas as pd

In [2]:
def create_dummy_columns(df, column_name):
    return pd.get_dummies(df, columns=[column_name], prefix=column_name)

In [8]:
portfolio = pd.read_json('../data/portfolio.json', orient='records', lines=True)
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [9]:
# Convert offer_type to dummies
portfolio = create_dummy_columns(portfolio,"offer_type")
portfolio.head()

,reward,channels,difficulty,duration,id,offer_type_bogo,offer_type_discount,offer_type_informational
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,True,False,False
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,True,False,False
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,False,False,True
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,True,False,False
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,False,True,False


In [10]:
# First, explode the array column to get one row per channel
channels_dummies = pd.get_dummies(portfolio['channels'].explode(), prefix='channel')

# Then aggregate back to get one row per original record
channels_dummies = channels_dummies.groupby(level=0).max()

# Join the dummy columns back to the original dataframe
portfolio = portfolio.join(channels_dummies)

portfolio.head()

,reward,channels,difficulty,duration,id,offer_type_bogo,offer_type_discount,offer_type_informational,channel_email,channel_mobile,channel_social,channel_web
0,10,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,True,False,False,True,True,True,False
1,10,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,True,False,False,True,True,True,True
2,0,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,False,False,True,True,True,False,True
3,5,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,True,False,False,True,True,False,True
4,5,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,False,True,False,True,False,False,True
